In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
import sklearn.datasets
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px

pd.options.display.max_columns = 150
%matplotlib inline

### Анализ данных
Сборник SMS-спама - это набор SMS-сообщений с тегами, собранных для исследования SMS-спама. Он содержит один набор SMS-сообщений на английском языке из 5 574 сообщений, помеченных как «легитимные» или «спам».

файлы содержат по одному сообщению в строке. Каждая строка состоит из двух столбцов: v1 содержит метку (ветчина или спам), а v2 содержит необработанный текст.

Этот корпус был получен из бесплатных или бесплатных источников для исследований в Интернете:

-> Коллекция из 425 спам-сообщений SMS была вручную извлечена с веб-сайта Grumbletext. Это британский форум, на котором пользователи сотовых телефонов публично заявляют о спам-сообщениях в виде SMS, причем большинство из них не сообщает о полученном спаме. Идентификация текста спам-сообщений в претензиях - очень сложная и трудоемкая задача, требующая тщательного сканирования сотен веб-страниц. Веб-сайт Grumbletext: [Веб-ссылка] .
-> Подмножество из 3 375 SMS, случайно выбранных любительских сообщений из NUS SMS Corpus (NSC), который представляет собой набор данных из примерно 10 000 легитимных сообщений, собранных для исследований в Департаменте компьютерных наук Национального университета Сингапура. Сообщения в основном исходят от сингапурцев и в основном от студентов, обучающихся в университете. Эти сообщения были получены от добровольцев, которые были осведомлены о том, что их вклад станет общедоступным. Корпус NUS SMS доступен по адресу: [Web Link] .
-> Список из 450 радиолюбительских SMS-сообщений, собранных из докторской диссертации Кэролайн Тэг, доступен на [Web Link] .
-> Наконец, мы добавили корпус SMS Spam Corpus v.0.1 Big. Он содержит 1 002 любительских SMS-сообщения и 322 спам-сообщения, и он общедоступен по адресу: [Web Link] . Этот корпус использовался в следующих академических исследованиях:

In [ ]:
df_ = pd.read_csv(r"C:\Users\nikita.saprykin\Desktop\Машинное обучение в бизнесе\new\spam.csv",  encoding='ISO-8859-1')
df_.head(15)

In [ ]:
df_.describe().T

In [ ]:
df_.shape

#### Пропущенные значения

In [ ]:
df_.isna().sum()

In [ ]:
import missingno as msno
msno.bar(df_, color = '#6389df', figsize = (6,4))  

In [ ]:
df_ = df_[['v2','v1']]
df_.columns = ['sms','spam']
df=pd.DataFrame()
mapping = {'spam': 1,'ham': 0}
df['sms']=df_['sms']
df['spam']=df_['spam'].map(mapping)

df.head(5)

### Очистка данных

In [ ]:
def remove_duplicate(data):
    
    print("До удаления дубликатов кол-во строк = ",df.shape[0])
    data.drop_duplicates(keep="first", inplace=True) 
    print("После удаления дубликатов кол-во строк = ",df.shape[0])
    return "Проверка дубликатов"

remove_duplicate(df)

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df)

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df)

### Test train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['spam'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [ ]:
features = df['sms']
target = df['spam']

In [ ]:
#combine
sms = Pipeline([
                ('imputer', TextImputer('sms', '')),
                ('selector', ColumnSelector(key='sms')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('sms', sms),
                      ])

In [ ]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)
#Посмотрим, как выглядит наш pipeline
pipeline.steps

In [ ]:
with open("pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

### Проверка работоспособности АПИ

### клонирование репозитория и создание его образа

 $ git clone https://github.com/filinon205/project_1.git

 $ cd flack_docker/

 $ sudo docker build -t nikita/flack_docker .

### Запуск контейнера к которому необходимо обращаться через АПИ

 $ docker run -d -p 8180:8180 -p 8181:8181 -v C:/Users/nikita.saprykin/Desktop/Машинное обучение в бизнесе/flack_docker/app/models:/app/app/models nikita/flack_docker/pipeline.dill


### Проверка работоспособности и качества пайплайна
roc_auc_score  98%

In [11]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [12]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [13]:
X_test.head(3)

,sms,is_spam
0,"Funny fact Nobody teaches volcanoes 2 erupt, t...",0
1,I sent my scores to sophas and i had to do sec...,0
2,We know someone who you know that fancies you....,1


In [14]:
with open('pipeline.dill', 'rb') as in_strm:
#with open('"app/models/my_1st_pipeline.dill"', 'rb') as in_strm:
    pipeline = dill.load(in_strm)
    
predictions = pipeline.predict_proba(X_test)
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [15]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test)

0.9796411318150448

In [16]:
0.9851996879407087

0.9851996879407087

In [17]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [18]:
X_test[['sms']].head(3)

,sms
0,"Funny fact Nobody teaches volcanoes 2 erupt, t..."
1,I sent my scores to sophas and i had to do sec...
2,We know someone who you know that fancies you....


In [19]:
import urllib.request
import json      


def get_prediction(x):
    sms = x
    body = {'sms': sms, 
            } 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['is_spam']

In [20]:
#Проверка
get_prediction('Your question this week will enter u in our draw 4 cash.')

0.7948225582927236

In [21]:
# тестовый массив
predictions = X_test['sms'].apply(lambda x: get_prediction(x), 1)

In [22]:
# массив
predictions

0       0.312016
1       0.005498
2       0.816170
3       0.003577
4       0.999995
          ...   
1834    0.000263
1835    0.001471
1836    0.013453
1837    0.372728
1838    0.965014
Name: sms, Length: 1839, dtype: float64

In [23]:
# Качество
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.9796411318150448